## 기본 Setting

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=619625 sha256=5b83ba41919ff7cbdbe3b0a271b30553dc35441330dfbbd1386fe0772a7fa29a
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2ahl8vay
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2ahl8vay
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 881.9/881.9 MB 77.6 MB/s eta 0:00:01tcmalloc: large alloc 1102397440 bytes == 0x384e4000 @  0x7fda195f5615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x5

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


## 데이터셋 전처리 진행

In [8]:
import pandas as pd
data = pd.read_excel("한국어_단발성_대화_데이터셋.xlsx")

In [9]:
data

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [10]:
data.loc[(data["Emotion"]== "공포"), "Emotion"] = 0
data.loc[(data["Emotion"]== "놀람"), "Emotion"] = 1
data.loc[(data["Emotion"]== "분노"), "Emotion"] = 2
data.loc[(data["Emotion"]== "슬픔"), "Emotion"] = 3
data.loc[(data["Emotion"]== "중립"), "Emotion"] = 4
data.loc[(data["Emotion"]== "행복"), "Emotion"] = 5
data.loc[(data["Emotion"]== "혐오"), "Emotion"] = 6

data_list = list()
for sentence, emotion in zip(data["Sentence"], data["Emotion"]):
  data_list.append([sentence,emotion])

In [11]:
data_list[:10]

[['언니 동생으로 부르는게 맞는 일인가요..??', 0],
 ['그냥 내 느낌일뿐겠지?', 0],
 ['아직너무초기라서 그런거죠?', 0],
 ['유치원버스 사고 낫다던데', 0],
 ['근데 원래이런거맞나요', 0],
 [' 남자친구가 떠날까봐요', 0],
 ['이거 했는데 허리가 아플수도 있나요? ;;', 0],
 ['내가불안해서꾸는걸까..', 0],
 [' 일주일도 안 남았당...ㅠㅠ', 0],
 ['약은 최대한 안먹으려고 하는데좋은 음시있나요?0', 0]]

## Train data , Test data 분리

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_set, test_set = train_test_split(data_list, test_size=0.25, random_state=0) # train : test = 4 : 1

In [14]:
len(train_set), len(test_set)

(28945, 9649)

In [15]:
train_set[0]

['그냥 멋져유 ~요리할땐 더 멋져유~기부했다고 들었을땐 더더더 멋져부러유~~^^', 5]

## KoBERT 입력 데이터 구성하기

BERT 입력값은 (token + segment + position embedding의 합으로 구성)  
https://happy-obok.tistory.com/23 여기서 확인!!

In [16]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):

    # sentence , label data를 BERT의 입력값에 맞게 변환하는 transformer를 생성
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)

    ## 생성한 transformer로 sentence를 변환하여 저장
    self.sentences = [transform([data[sent_idx]]) for data in dataset]
    self.labels = [np.int32(data[label_idx]) for data in dataset]
  
  def __getitem__ (self, i):
    return (self.sentences[i] + (self.labels[i], )) # 각 index에 맞는 item 반환 진행 --> 왜 이런 형태인지는 잘 모르겠음
  
  def __len__(self):
    return (len(self.labels))
    

In [23]:
# Parameter setting 진행
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
# Kobert 모듈에서 제공하는 get_tokenizer와 vocab를 활용해 tokneizer를 구성한다
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
data_train[0]

## 출력값 설명
## 1번 array - 1로 padding된 sequence - 총 길이 64 고정
## 2번 array - padding 전 sequnce 길이 - 42
## 3번 array - 어텐션 마스크 sequence (1로 padding이 된 부분은 어텐션 함수가 적용되지 않아도 됨을 알려주는 시퀀스)
##           - 일단은 전부 0으로 구성하고, 아래의 gen_attention_mask에서 필요한 부분만 1로 변경
## 마지막 숫자 - label값

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
        1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
        7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
         517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(42, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 5)

In [20]:
# Torch에 맞게 최종 입력 dataset 구성 

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## KoBert 다중 분류기를 위한 미세 조정 모델 만들기

In [21]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes=7, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    ## classifier는 선형 회귀 모델로 구성 (input size = 768, output size = 7 (label이 7개로 구성))
    self.classifier = nn.Linear(hidden_size, num_classes)

    ## overfitting 방지를 위한 dropout 비율 설정
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  # attention mask sequence를 구성해주는 함수 --> padding이 아닌 영역을 0에서 1로 변경
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i,v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    
    return attention_mask.float()
  
  # bert + classifier를 관통하는 forward 연산 진행
  def forward(self, token_ids, valid_length, segment_ids):

    # attention_mask 계산
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    # bert에 input 투입, 변수명이 pooler인거 보니 출력 embedding에 mean pooling 적용한 값이지 않을까 추측
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    # dropout 비율이 존재한다면, dropout 적용
    if self.dr_rate:
        out = self.dropout(pooler)

    # classifier 진행
    return self.classifier(out) 

In [22]:
# Kobert + classifier 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer (Adam), scheduler 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

## Classifier 학습 진행 (미세 조정)

In [24]:
# 설정한 epoch = 3
for e in range(num_epochs):

    train_acc = 0.0
    test_acc = 0.0
    model.train()

    # train set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # loss 는 CrossEntropyLoss를 이용 -> backpropagation 학습 진행
        loss = loss_fn(out, label)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # train data set 정확도 확인
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()

    # test set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        
        # test set 정확도 확인
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-24-26bd2ac989b5>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/453 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.0116946697235107 train acc 0.078125
epoch 1 batch id 201 loss 1.5046465396881104 train acc 0.2556747512437811
epoch 1 batch id 401 loss 1.2270387411117554 train acc 0.352322319201995
epoch 1 train acc 0.3671479353330736


<ipython-input-24-26bd2ac989b5>:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 2 test acc 0.5375346330585214


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.8812023997306824 train acc 0.65625
epoch 3 batch id 201 loss 1.0947364568710327 train acc 0.6090640547263682
epoch 3 batch id 401 loss 0.8195181488990784 train acc 0.630338216957606
epoch 3 train acc 0.6359218770289572


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 3 test acc 0.5505642654412758


## 학습된 Model로 결과 예측해보기

In [36]:
def predict(predict_sentence):

    # 1. data set 구성 (문장, 라벨)
    data = [predict_sentence, '0']
    dataset_another = [data]

    # 2. data를 bert의 입력에 맞게 변환하기
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # 모델 forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # torch out -> numpy 형식으로 변환
        test_eval=[]
        logits = out[0].detach().cpu().numpy()
        
        # 값이 가장 큰 인덱스를 출력
        if np.argmax(logits) == 0:
            test_eval= "공포가"
        elif np.argmax(logits) == 1:
            test_eval = "놀람이"
        elif np.argmax(logits) == 2:
            test_eval = "분노가"
        elif np.argmax(logits) == 3:
            test_eval = "슬픔이"
        elif np.argmax(logits) == 4:
            test_eval = "중립이"
        elif np.argmax(logits) == 5:
            test_eval = "행복이"
        elif np.argmax(logits) == 6:
            test_eval = "혐오가"
        print(">> 입력하신 내용에서 " + test_eval + " 느껴집니다.")

In [38]:
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "q":
      break
    
    predict(sentence)
    print()

하고싶은 말을 입력해주세요 : 나 무서워..


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 공포가 느껴집니다.

하고싶은 말을 입력해주세요 : 너 지금 납치된거야.. ㅎ
>> 입력하신 내용에서 공포가 느껴집니다.

하고싶은 말을 입력해주세요 : 으 극혐쓰..
>> 입력하신 내용에서 분노가 느껴집니다.

하고싶은 말을 입력해주세요 : 오늘 날씨 너무 좋은데?
>> 입력하신 내용에서 놀람이 느껴집니다.

하고싶은 말을 입력해주세요 : q


In [40]:
torch.save(model, "model.pth")